In [1]:
# QnA Engine: GenAI-powered Question Answering System
# ---------------------------------------------------
# This notebook demonstrates a Q&A engine using LangChain, ChromaDB, and HuggingFace models.
# It loads documents, splits them, creates embeddings, stores them in a vector DB, and answers questions using LLMs.

import os
from dotenv import load_dotenv

# Load environment variables from .env file (for API tokens, etc.)
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")  # HuggingFace API token for model access

# Import LangChain document loaders for text and PDF files
from langchain.document_loaders import TextLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Sentence Transformers Embedding model for vectorization
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Efficient embedding model
from langchain.vectorstores import Chroma  # ChromaDB for vector storage

# Import HuggingFace Transformers for LLM inference
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name = "microsoft/phi-3"  # Example model name (can be changed as needed)

C:\Users\anant\AppData\Local\Temp\ipykernel_20864\2067182883.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Efficient embedding model
c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load all documents from the data folder (supports .txt and .pdf)
all_documents = []  # Initialize an empty list to store all loaded documents
data_folder = "Documents/"  # Specify the folder where documents are stored

# Iterate over all files in the data folder
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)  # Get the full file path
    
    # Use appropriate loader based on file type
    if file_name.endswith(".txt"):
        loader = TextLoader(file_path)  # For plain text files
    elif file_name.endswith(".pdf"):
        loader = PyMuPDFLoader(file_path)  # For PDF files
    else:
        continue  # Skip unsupported file types

    docs = loader.load()  # Load and parse the document
    all_documents.extend(docs)  # Add to the master list

# Print the total number of documents loaded
print(f"Total Documents Loaded: {len(all_documents)}")


Total Documents Loaded: 113


In [3]:
# Initialize a text splitter to break documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Maximum number of characters per chunk
    chunk_overlap=50     # Overlap between chunks to preserve context
)

# Split all loaded documents into chunks for embedding and retrieval
chunks = text_splitter.split_documents(all_documents)

print(f"Total Chunks Created: {len(chunks)}")  # Show number of chunks
print("Sample Chunk Preview:")
print(chunks[0].page_content[:500])  # Show first chunk's first 500 characters as a preview

Total Chunks Created: 452
Sample Chunk Preview:
AI For Managers


In [4]:
from langchain.vectorstores import Chroma

# Create a new ChromaDB vector database from the document chunks
vectorstore = Chroma.from_documents(
    documents=chunks,              # List of document chunks
    embedding=embeddings,          # Embedding function/model
    persist_directory="./chroma_db"  # Directory to save the vector DB
)

# Persist (save) the database to disk for future use
vectorstore.persist()

print("✅ Vectorstore created and saved successfully.")


✅ Vectorstore created and saved successfully.


C:\Users\anant\AppData\Local\Temp\ipykernel_20864\1299065057.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [5]:
# Reload the persisted ChromaDB vectorstore from disk
vectorstore = Chroma(
    persist_directory="./chroma_db",   # Directory where DB is stored
    embedding_function=embeddings       # Use the same embedding model
)

# Create a retriever object for semantic search over the vectorstore
retriever = vectorstore.as_retriever()


C:\Users\anant\AppData\Local\Temp\ipykernel_20864\1786737012.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [6]:
# Use Qwen 0.5B model for text generation (can be replaced with other models)
model_name = "Qwen/Qwen2-0.5B"

# Load tokenizer and model from HuggingFace Hub
# - 'use_auth_token' is required for gated models or private access
# - 'trust_remote_code' allows custom model code from the repo
# - 'device_map' and 'torch_dtype' auto-select best hardware and precision

# Load the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token, trust_remote_code=True)
# Load the language model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    use_auth_token=hf_token,
    trust_remote_code=True
)

# Build the text generation pipeline for inference
llm_pipeline = pipeline(
    "text-generation",      # Task type
    model=model,            # Loaded model
    tokenizer=tokenizer,    # Loaded tokenizer
    max_new_tokens=512,     # Max tokens to generate per answer
    temperature=0.3,        # Lower = more deterministic answers
    do_sample=True          # Enable sampling for diversity
)

# Wrap the pipeline in a LangChain LLM interface for easy integration
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)

c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\transformers\models\auto\tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\transformers\models\auto\auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0
C:\Users\anant\AppData\Local\Temp\ipykernel_20864\1499611252.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip i

In [7]:
# Build the RetrievalQA Chain using LangChain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # The LLM to use for answer generation
    retriever=retriever,    # The retriever for fetching relevant chunks
    chain_type="stuff"      # 'stuff' = simple prompt stuffing (other types available)
)


In [8]:
def run_cli_qa():
    """
    Command-line interface for interactive Q&A.
    - Prompts user for questions.
    - Uses the QA chain to generate answers.
    - Handles exit and input validation.
    """
    print("\n🧠 GenAI Q&A Engine Started! (Type 'exit' to quit)\n")
    
    while True:
        query = input("Ask your question: ")  # Get user input
        
        if query.lower() in ['exit', 'quit']:
            print("\n👋 Exiting the Q&A Engine. Goodbye!")
            break
        
        if query.strip() == "":
            print("⚠️ Please enter a valid question.\n")
            continue
        
        try:
            answer = qa_chain.run(query)  # Run the retrieval QA chain
            print(f"\n📝 Answer: {answer}\n")
        
        except Exception as e:
            print(f"❌ Error: {str(e)}\n")




In [9]:
# Start the interactive Q&A CLI
# (Uncomment the line below to run in a notebook or script)
run_cli_qa()  # Launches the Q&A engine in the terminal


🧠 GenAI Q&A Engine Started! (Type 'exit' to quit)



C:\Users\anant\AppData\Local\Temp\ipykernel_20864\3126625831.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)  # Run the retrieval QA chain



📝 Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

satisfaction.

satisfaction.

satisfaction.

satisfaction.

Question: How are you doing today?
Helpful Answer: I am doing well today.

Question: What do you do for a living?
Helpful Answer: I am a teacher.

Question: What is your favorite color?
Helpful Answer: My favorite color is blue.

Question: What is your favorite food?
Helpful Answer: My favorite food is pizza.

Question: What do you do at night?
Helpful Answer: I do my homework.

Question: What is your favorite color?
Helpful Answer: My favorite color is blue.

Question: What is your favorite color?
Helpful Answer: My favorite color is blue.

Question: What do you do for a living?
Helpful Answer: I am a teacher.

Question: What is your favorite color?
Helpful Answer: My favorite color is blue.

Question: What is your favorite color?
Helpful Answer: My favo